In [1]:
from pyscal.core import System
import json
import yaml
import numpy as np
from json import JSONEncoder

In [2]:
class NumpyArrayEncoder(JSONEncoder):
    """
    Encode numpy to dump in json
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyArrayEncoder, self).default(obj)
    
def get_angle(vec1, vec2):
    """
    Get angle between two vectors in degrees
    
    Parameters
    ----------
    vec1: list
        first vector
    
    vec2: list
        second vector
    
    Returns
    -------
    angle: float
        angle in degrees
    
    Notes
    -----
    Angle is rounded to two decimal points
    
    """
    return np.round(np.arccos(np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))*180/np.pi, decimals=2)

def write_file(outfile, data):
    """
    Write a given dict as json file
    
    Parameters
    ----------
    outfile: string
        name of output file. `.json` will be added to the given file name
    
    data: dict
        input data dict
    
    Returns
    -------
    None
    """
    with open(".".join([outfile, "json"]), "w") as fout:
        json.dump(convert_to_dict(sys), fout, cls=NumpyArrayEncoder)
    #with open(".".join([outfile, "yaml"]), "w") as fout:
    #    yaml.unsafe_dump(convert_to_dict(sys), fout)
        
def convert_to_dict(sys):
    """
    Convert a pyscal System object to data dictionary
    
    Parameters
    ----------
    sys: pyscal System
        input system
    
    Returns
    -------
    info: dict
        dict with parsed information
    """
    info = {}
    
    #not available
    info["latticeparameter"] = None
    
    #needs to be defined
    info["Occupancy"] = sys.atoms.species
    info["Element"] = np.unique(sys.atoms.species)
    info["CellVolume"] = sys.volume
    info["NumberOfAtoms"] = sys.natoms
    
    #available 
    info["FirstAxisComponent"] = sys.box[0]
    info["SecondAxisComponent"] = sys.box[1]
    info["ThirdAxisComponent"] = sys.box[2]
    info["LatticeParameterLengthA"] = sys.boxdims[0]
    info["LatticeParameterLengthB"] = sys.boxdims[1]
    info["LatticeParameterLengthC"] = sys.boxdims[2]
    
    info["LatticeParameterAngleAlpha"] = get_angle(sys.box[0], sys.box[1])
    info["LatticeParameterAngleBeta"] = get_angle(sys.box[1], sys.box[2])
    info["LatticeParameterAngleGamma"] = get_angle(sys.box[2], sys.box[0])
    
    info["X_AxisCoordinate"] = np.array(sys.atoms.positions)[:,0]
    info["Y_AxisCoordinate"] = np.array(sys.atoms.positions)[:,1]
    info["Z_AxisCoordinate"] = np.array(sys.atoms.positions)[:,2]
    
    return info

In [3]:
sys = System()
sys.read_inputfile("al_data/Al.poscar", format="poscar")

In [4]:
convert_to_dict(sys)
write_file("dump", convert_to_dict(sys))